In [13]:
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import urllib
import os
import pandas as pd
from selenium import webdriver
from time import sleep
import re
from selenium.webdriver.common.keys import Keys

In [19]:
def page_scrap (job_url):
    
    driver = webdriver.Chrome(executable_path="/Users/yesplum/desktop/GApro/chromedriver/chromedriver")
    driver.get(job_url)
    sleep(5)
    html = driver.page_source
    sleep(3)
    driver.close()
    soup = BeautifulSoup(html, 'lxml', from_encoding="utf-8")
    
    job_title = []
    company = []
    job_id = []
    seniority = []
    job_cat = []
    salary = []
    requirements = []
    post_date = []

    for entry in soup.find_all('h1', {'id': "job_title"}):
        job_title.append(entry.renderContents())

    for entry in soup.find_all('p', {'name': "company"}):
        company.append(entry.renderContents())

    for entry in soup.find_all('span', {'class': "black-60 db f6 fw4 mv1"}):
        job_id.append(entry.renderContents())
    
    for entry in soup.find_all('p', {'id': "seniority"}):
        seniority.append(entry.renderContents())

    for entry in soup.find_all('p', {'id': "job-categories"}):
        job_cat.append(entry.renderContents()) 

    for entry in soup.find_all('span', {'id': "last_posted_date"}):
        post_date.append(entry.renderContents())     

    salary_all = soup.find_all("span", attrs={'class': 'dib'})
    list_of_inner_text = [x.text for x in salary_all]
    salary_text = '/ '.join(list_of_inner_text)
    salary.append(salary_text)

    requirement_all = soup.find_all("div", attrs={'id': 'requirements'})
    list_of_inner_text = [x.text for x in requirement_all]
    requirements_text = ', '.join(list_of_inner_text)
    requirements.append(requirements_text)

    
    data = pd.DataFrame(columns = ['Job_Id','Job_Title','Company','Date_Posted','Salary',
                                   'Seniority','Category','Requirements'])

    data = data.append({'Job_Id':job_id,'Job_Title':job_title,'Company':company,
                    'Date_Posted':post_date,'Salary':salary,'Seniority':seniority,
                    'Category':job_cat,'Requirements':requirements},ignore_index=True) 

    return data

In [20]:
driver = webdriver.Chrome(executable_path="/Users/yesplum/desktop/GApro/chromedriver/chromedriver")
driver.get("https://www.mycareersfuture.sg/")
elem = driver.find_element_by_name("search-text")
elem.send_keys("data") #job search keyword
elem.send_keys(Keys.RETURN)
sleep(3)
current_url = driver.current_url
sleep(3)
driver.close()

y=0
links = []

for page in range(2):
    
    new_next_url = current_url.replace('page=' + str(y),'page=' +str(page))
    #print(new_next_url)

    driver2 = webdriver.Chrome(executable_path="/Users/yesplum/desktop/GApro/chromedriver/chromedriver")
    driver2.get(new_next_url)
    sleep(3)
    html1 = driver2.page_source 
    driver2.close()

    soup = BeautifulSoup(html1, 'lxml', from_encoding="utf-8")

    for entry in soup.find_all("a", attrs={'class': re.compile(r'JobCard')}):
        link = entry.get('href')
        links.append(link)

In [21]:
job_df = pd.DataFrame()

for i in range (len(links)):
    job_url  =  "https://www.mycareersfuture.sg%s" % links[i]
    temp_df = page_scrap(job_url)
    job_df = job_df.append(temp_df, ignore_index=True)

,Job_Id,Job_Title,Company,Date_Posted,Salary,Seniority,Category,Requirements
0,[b' JOB-2019-0088700 '],"[b'Manager, Clinical Data Management']",[b'TESSA THERAPEUTICS LTD.'],[b'Posted 25 Apr 2019'],[/ Salary undisclosed/ / Closing on 25 May 2019],[b'Manager'],[b'Healthcare / Pharmaceutical'],[RequirementsRequirements Bachelor’s or Maste...
1,[b' JOB-2019-0088666 '],[b'Senior Data Analyst'],[b'PROPERTYGURU PTE. LTD.'],[b'Posted 25 Apr 2019'],"[/ $6,000to$8,000/ $6,000/ to$8,000/ Monthly/ ...","[b'Professional, Senior Executive']","[b'Engineering, Information Technology']",[Requirements Bachelor’s degree in IT or relev...
2,[b' JOB-2019-0088485 '],[b'Senior Level Data Engineer'],[b'TRAVELOKA SERVICES PTE. LTD.'],[b'Posted 25 Apr 2019'],"[/ $8,300to$15,000/ $8,300/ to$15,000/ Monthly...",[b'Senior Executive'],[b'Information Technology'],"[Requirements Passion in big data, software e..."
3,[b' JOB-2019-0058738 '],[b'Data Analyst'],[b'INNOSPIRE PTE. LTD.'],[b'Posted 25 Apr 2019'],[/ Salary undisclosed/ / Closing on 25 May 2019],[b'Senior Executive'],[b'Information Technology'],"[Requirements Degree in Mathematics, Economi..."
4,[b' JOB-2019-0088798 '],[b'Senior SAP\xc2\xae Data Migration Consultant'],[b'CPM CONSULTING PTE. LTD.'],[b'Posted 25 Apr 2019'],"[/ $6,000to$8,000/ $6,000/ to$8,000/ Monthly/ ...",[b'Professional'],[b'Consulting '],[RequirementsBase Location: Singapore Qualif...
5,[b' JOB-2019-0088223 '],"[b'Data Centre Sales Director, Asia']",[b'GEORGE FISCHER PTE LTD'],[b'Posted 25 Apr 2019'],[/ Salary undisclosed/ / Closing on 25 May 2019],[b'Senior Management'],"[b'Engineering, Manufacturing']","[Requirements Positive attitude, motivation fo..."
6,[b' JOB-2019-0088175 '],[b'TEMP (RESEARCH AND DATA) \xe2\x80\x93 MAY T...,[b'Ministry of Communications and Information'],[b'Posted 25 Apr 2019'],[/ Salary undisclosed/ / Closing on 01 May 2019],[b''],[b'Public / Civil Service'],[]
7,[b' JOB-2019-0088262 '],[b'Data Migration Manager'],[b'STANDARD CHARTERED BANK (SINGAPORE) LIMITED'],[b'Posted 25 Apr 2019'],"[/ $8,000to$14,999/ $8,000/ to$14,999/ Monthly...",[b'Manager'],"[b'Banking and Finance, Information Technology']",[RequirementsOur Ideal Candidate - Minimum 8 ...
8,[b' JOB-2019-0088577 '],[b'Senior Level Data Scientist'],[b'TRAVELOKA SERVICES PTE. LTD.'],[b'Posted 25 Apr 2019'],"[/ $8,300to$15,000/ $8,300/ to$15,000/ Monthly...",[b'Executive'],[b'Information Technology'],[Requirements Solid programming skills and un...
9,[b' JOB-2019-0088557 '],[b'Mid - Senior Level Data Scientist'],[b'TRAVELOKA SERVICES PTE. LTD.'],[b'Posted 25 Apr 2019'],"[/ $6,100to$10,700/ $6,100/ to$10,700/ Monthly...",[b'Executive'],[b'Information Technology'],[Requirements Solid programming skills and un...
